In [339]:
import json
from pathlib import Path
from ast import literal_eval

In [340]:
ROOT_DIR = Path('notebooks/format_data.ipynb').resolve().parents[2]
DATA_DIR = ROOT_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PREPARED_DIR = DATA_DIR / "prepared"

In [4]:
with open(RAW_DIR / "CENTCOM/documents.json", "r") as fp:
    lines = fp.readlines()

In [5]:
lines[0]

'{ "_id" : "001C9C3F3DFE16B4921B1E906F66E161", "sourceName" : "CENTCOM", "sourceUrl" : "http://www.centcom.mil/MEDIA/PRESS-RELEASES/Press-Release-View/Article/904608/centcom-reinforces-support-for-syrian-arab-coalition/", "wordCount" : 193, "sentenceCount" : 8, "title" : "CENTCOM reinforces support for Syrian Arab Coalition", "text" : "Photographs of U.S. service members in Syria shown wearing patches associated with the Kurdish YPG have recently circulated through traditional and social media. To clarify: U.S. forces are advising and assisting Syrian Democratic Forces (SDF), primarily Syrian Arab forces operating in the northern part of the country who are leading efforts in the campaign to defeat Daesh.\\n\\nGen. Joseph Votel, commander of U.S. Central Command said, \\"We\'ve been very clear from the outset of this campaign that to defeat Daesh we would need to work with many different elements operating on the ground in Iraq and Syria. In Syria, with the support of the 60-plus natio

In [6]:
type(lines[0])

str

In [7]:
literal_eval(lines[0])

{'_id': '001C9C3F3DFE16B4921B1E906F66E161',
 'sourceName': 'CENTCOM',
 'sourceUrl': 'http://www.centcom.mil/MEDIA/PRESS-RELEASES/Press-Release-View/Article/904608/centcom-reinforces-support-for-syrian-arab-coalition/',
 'wordCount': 193,
 'sentenceCount': 8,
 'title': 'CENTCOM reinforces support for Syrian Arab Coalition',
 'text': 'Photographs of U.S. service members in Syria shown wearing patches associated with the Kurdish YPG have recently circulated through traditional and social media. To clarify: U.S. forces are advising and assisting Syrian Democratic Forces (SDF), primarily Syrian Arab forces operating in the northern part of the country who are leading efforts in the campaign to defeat Daesh.\n\nGen. Joseph Votel, commander of U.S. Central Command said, "We\'ve been very clear from the outset of this campaign that to defeat Daesh we would need to work with many different elements operating on the ground in Iraq and Syria. In Syria, with the support of the 60-plus nation Count

In [8]:
type(literal_eval(lines[0]))

dict

In [9]:
import spacy

nlp = spacy.load("en_core_web_lg")

my_dict = literal_eval(lines[0])

doc = nlp(my_dict["text"])
for token in doc:
    print(token.text)

Photographs
of
U.S.
service
members
in
Syria
shown
wearing
patches
associated
with
the
Kurdish
YPG
have
recently
circulated
through
traditional
and
social
media
.
To
clarify
:
U.S.
forces
are
advising
and
assisting
Syrian
Democratic
Forces
(
SDF
)
,
primarily
Syrian
Arab
forces
operating
in
the
northern
part
of
the
country
who
are
leading
efforts
in
the
campaign
to
defeat
Daesh
.



Gen.
Joseph
Votel
,
commander
of
U.S.
Central
Command
said
,
"
We
've
been
very
clear
from
the
outset
of
this
campaign
that
to
defeat
Daesh
we
would
need
to
work
with
many
different
elements
operating
on
the
ground
in
Iraq
and
Syria
.
In
Syria
,
with
the
support
of
the
60
-
plus
nation
Counter
-
Daesh
Coalition
,
the
SDF
,
which
includes
the
Syrian
Arab
Coalition
,
has
made
extraordinary
progress
over
the
past
20
-
plus
months
,
retaking
more
than
18,000
square
kilometers
of
territory
previously
held
by
the
enemy
.
We
will
continue
to
support
their
efforts
specific
to
the
fight
against
Daesh
.
At
the
same
t

In [62]:
def prepare_document_json(raw_dir: Path, doc_path: Path, output_dir: Path) -> None:
    """Prepare re3d dataset document json files into one json object

    Args:
        raw_dir (Path): Directory with raw json files
        doc_path (Path): file path from raw_dir
        output_dir (Path): Output Directory
    """
    with open(raw_dir / doc_path, "r") as fp:
        lines = fp.readlines()

    lines_dict = {literal_eval(line)["_id"]:literal_eval(line) for line in lines}

    # make output dir if not exist
    sub_dir = output_dir / doc_path.parents[0]
    sub_dir.mkdir(parents=True, exist_ok=True)

    with open(output_dir / doc_path, "w") as fp:
        json.dump(lines_dict, fp)

In [63]:
prepare_document_json(RAW_DIR, Path("CENTCOM/documents.json"), PREPARED_DIR)

In [384]:
def make_dataset():
    doc_jsons = list(RAW_DIR.glob("**/documents.json"))
    for doc_json in doc_jsons:
        with open(doc_json, "r") as fp:
            docs = [literal_eval(line) for line in fp]

        for document in docs:
            ents_path = doc_json.parent / "entities.json"
            with open(ents_path, "r") as fp:
                ents = [literal_eval(line) for line in fp]
            docid = document["_id"]
            doc_ents = [ent for ent in ents if ent["documentId"] == docid]

            token_spans = get_token_spans(document["text"])
            words = label_words(token_spans)
            words = bio_tagger(words)
            df = pd.DataFrame(words)
        

In [385]:
make_dataset()

NameError: name 'label_words' is not defined

In [383]:
Path('/home/enyquist/repos/RLNER/data/raw/CENTCOM/documents.json').parent

PosixPath('/home/enyquist/repos/RLNER/data/raw/CENTCOM')

In [365]:
doc_jsons = list(RAW_DIR.glob("**/documents.json"))

In [375]:
with open(doc_jsons[0], "r") as fp:
    docs = [literal_eval(line) for line in fp]

In [378]:
for doc in docs:
    docid = doc["_id"]
    doc_ents = [ent for ent in ents if ent["documentId"] == docid]

In [379]:
doc_ents

[{'_id': 'FEB7BC61C9765C9859F46B9B634F039F-0-0-31-MilitaryPlatform',
  'begin': 0,
  'end': 31,
  'type': 'MilitaryPlatform',
  'value': 'Air Force F-16 Fighting Falcons',
  'documentId': 'FEB7BC61C9765C9859F46B9B634F039F',
  'confidence': 0.96},
 {'_id': 'FEB7BC61C9765C9859F46B9B634F039F-0-0-9-Organisation',
  'begin': 0,
  'end': 9,
  'type': 'Organisation',
  'value': 'Air Force',
  'documentId': 'FEB7BC61C9765C9859F46B9B634F039F',
  'confidence': 0},
 {'_id': 'FEB7BC61C9765C9859F46B9B634F039F-0-122-162-Organisation',
  'begin': 122,
  'end': 162,
  'type': 'Organisation',
  'value': 'the Islamic State of Iraq and the Levant',
  'documentId': 'FEB7BC61C9765C9859F46B9B634F039F',
  'confidence': 1},
 {'_id': 'FEB7BC61C9765C9859F46B9B634F039F-0-126-162-Organisation',
  'begin': 126,
  'end': 162,
  'type': 'Organisation',
  'value': 'Islamic State of Iraq and the Levant',
  'documentId': 'FEB7BC61C9765C9859F46B9B634F039F',
  'confidence': 0},
 {'_id': 'FEB7BC61C9765C9859F46B9B634F039F-

In [371]:
ent_jsons = list(RAW_DIR.glob("**/entities.json"))

In [372]:
with open(ent_jsons[0], "r") as fp:
    ents = [literal_eval(line) for line in fp]

In [373]:
ents

[{'_id': '001C9C3F3DFE16B4921B1E906F66E161-0-15-35-Organisation',
  'begin': 15,
  'end': 35,
  'type': 'Organisation',
  'value': 'U.S. service members',
  'documentId': '001C9C3F3DFE16B4921B1E906F66E161',
  'confidence': 1},
 {'_id': '001C9C3F3DFE16B4921B1E906F66E161-0-39-44-Location',
  'begin': 39,
  'end': 44,
  'type': 'Location',
  'value': 'Syria',
  'documentId': '001C9C3F3DFE16B4921B1E906F66E161',
  'confidence': 1},
 {'_id': '001C9C3F3DFE16B4921B1E906F66E161-0-83-98-Organisation',
  'begin': 83,
  'end': 98,
  'type': 'Organisation',
  'value': 'the Kurdish YPG',
  'documentId': '001C9C3F3DFE16B4921B1E906F66E161',
  'confidence': 1},
 {'_id': '001C9C3F3DFE16B4921B1E906F66E161-0-87-94-Nationality',
  'begin': 87,
  'end': 94,
  'type': 'Nationality',
  'value': 'Kurdish',
  'documentId': '001C9C3F3DFE16B4921B1E906F66E161',
  'confidence': 0},
 {'_id': '001C9C3F3DFE16B4921B1E906F66E161-1-12-23-Organisation',
  'begin': 174,
  'end': 185,
  'type': 'Organisation',
  'value': 'U

In [90]:
first = list(doc_jsons)

[PosixPath('/home/enyquist/repos/RLNER/data/raw/CENTCOM/documents.json'),
 PosixPath('/home/enyquist/repos/RLNER/data/raw/UK Government/documents.json'),
 PosixPath('/home/enyquist/repos/RLNER/data/raw/US State Department/documents.json'),
 PosixPath('/home/enyquist/repos/RLNER/data/raw/Australian Department of Foreign Affairs/documents.json'),
 PosixPath('/home/enyquist/repos/RLNER/data/raw/Wikipedia/documents.json'),
 PosixPath('/home/enyquist/repos/RLNER/data/raw/BBC Online/documents.json'),
 PosixPath('/home/enyquist/repos/RLNER/data/raw/Delegation of the European Union to Syria/documents.json')]

In [91]:
for doc_json in RAW_DIR.glob("**/documents.json"):
    sub_path = Path("/".join(doc_json.parts[-2:]))
    prepare_document_json(RAW_DIR, Path(sub_path), PREPARED_DIR)

In [82]:
list(first.parents)

[PosixPath('/home/enyquist/repos/RLNER/data/raw/CENTCOM'),
 PosixPath('/home/enyquist/repos/RLNER/data/raw'),
 PosixPath('/home/enyquist/repos/RLNER/data'),
 PosixPath('/home/enyquist/repos/RLNER'),
 PosixPath('/home/enyquist/repos'),
 PosixPath('/home/enyquist'),
 PosixPath('/home'),
 PosixPath('/')]

In [87]:
Path("/".join(first.parts[-2:]))

PosixPath('CENTCOM/documents.json')

In [317]:
entity = { "_id" : "09FF200CBB76594D688F8EB565717543-0-0-28-Person", "begin" : 0, "end" : 28, "type" : "Person", "value" : "Minister for the Middle East", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 }

In [318]:
document = { "_id" : "09FF200CBB76594D688F8EB565717543", "sourceName" : "UK Government", "sourceUrl" : "https://www.gov.uk/government/news/minister-for-the-middle-east-condemns-attack-in-baghdad", "wordCount" : 95, "sentenceCount" : 5, "title" : "Minister for the Middle East condemns attack in Baghdad", "text" : "Minister for the Middle East, Tobias Ellwood, responds to the attack in the Karrada district of Baghdad\n\nForeign Office Minister, Tobias Ellwood, said:\n\nThe attack was a horrific act of terror against innocent people in the early hours of Sunday morning in Karrada, Baghdad. I offer my sincere condolences to the families and friends of the victims, and hope those injured in the attack recover quickly.\n\nDaesh targeted families out shopping for Eid and those celebrating suhur. It violated the peace of Ramadan. The UK stands by Iraq to defeat Daesh and end this violence." }

In [319]:
document["text"]

'Minister for the Middle East, Tobias Ellwood, responds to the attack in the Karrada district of Baghdad\n\nForeign Office Minister, Tobias Ellwood, said:\n\nThe attack was a horrific act of terror against innocent people in the early hours of Sunday morning in Karrada, Baghdad. I offer my sincere condolences to the families and friends of the victims, and hope those injured in the attack recover quickly.\n\nDaesh targeted families out shopping for Eid and those celebrating suhur. It violated the peace of Ramadan. The UK stands by Iraq to defeat Daesh and end this violence.'

In [320]:
ent_text = document["text"][entity["begin"]:entity["end"]]
ent_text

'Minister for the Middle East'

In [321]:
entity["type"]

'Person'

In [322]:
entities = [{ "_id" : "09FF200CBB76594D688F8EB565717543-0-0-28-Person", "begin" : 0, "end" : 28, "type" : "Person", "value" : "Minister for the Middle East", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-0-105-128-Person", "begin" : 105, "end" : 128, "type" : "Person", "value" : "Foreign Office Minister", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-0-130-144-Person", "begin" : 130, "end" : 144, "type" : "Person", "value" : "Tobias Ellwood", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-0-17-28-Location", "begin" : 17, "end" : 28, "type" : "Location", "value" : "Middle East", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 0 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-0-224-245-Temporal", "begin" : 224, "end" : 245, "type" : "Temporal", "value" : "early hours of Sunday", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-0-257-264-Location", "begin" : 257, "end" : 264, "type" : "Location", "value" : "Karrada", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-0-266-273-Location", "begin" : 266, "end" : 273, "type" : "Location", "value" : "Baghdad", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-0-30-44-Person", "begin" : 30, "end" : 44, "type" : "Person", "value" : "Tobias Ellwood", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-0-72-103-Location", "begin" : 72, "end" : 103, "type" : "Location", "value" : "the Karrada district of Baghdad", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-0-96-103-Location", "begin" : 96, "end" : 103, "type" : "Location", "value" : "Baghdad", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 0 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-1-0-1-Person", "begin" : 275, "end" : 276, "type" : "Person", "value" : "I", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-1-79-83-Person", "begin" : 354, "end" : 358, "type" : "Person", "value" : "hope", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 0 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-2-0-5-Organisation", "begin" : 405, "end" : 410, "type" : "Organisation", "value" : "Daesh", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-2-15-23-Organisation", "begin" : 420, "end" : 428, "type" : "Organisation", "value" : "families", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-3-25-32-Temporal", "begin" : 504, "end" : 511, "type" : "Temporal", "value" : "Ramadan", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-4-0-6-Organisation", "begin" : 513, "end" : 519, "type" : "Organisation", "value" : "The UK", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-4-17-21-Location", "begin" : 530, "end" : 534, "type" : "Location", "value" : "Iraq", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 0 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-4-17-21-Organisation", "begin" : 530, "end" : 534, "type" : "Organisation", "value" : "Iraq", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 0.97 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-4-32-37-Organisation", "begin" : 545, "end" : 550, "type" : "Organisation", "value" : "Daesh", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 1 },
{ "_id" : "09FF200CBB76594D688F8EB565717543-4-4-6-Location", "begin" : 517, "end" : 519, "type" : "Location", "value" : "UK", "documentId" : "09FF200CBB76594D688F8EB565717543", "confidence" : 0 }]

In [323]:
from typing import List, Tuple

def get_token_spans(text: str) -> List[Tuple[str, int]]:
    doc = nlp(text)
    token_spans = []
    for token in doc:
        token_span = doc[token.i: token.i + 1]
        token_spans.append((token.text, token_span.start_char, token_span.end_char))

    return token_spans

In [324]:
token_spans = get_token_spans(document["text"])
token_spans

[('Minister', 0, 8),
 ('for', 9, 12),
 ('the', 13, 16),
 ('Middle', 17, 23),
 ('East', 24, 28),
 (',', 28, 29),
 ('Tobias', 30, 36),
 ('Ellwood', 37, 44),
 (',', 44, 45),
 ('responds', 46, 54),
 ('to', 55, 57),
 ('the', 58, 61),
 ('attack', 62, 68),
 ('in', 69, 71),
 ('the', 72, 75),
 ('Karrada', 76, 83),
 ('district', 84, 92),
 ('of', 93, 95),
 ('Baghdad', 96, 103),
 ('\n\n', 103, 105),
 ('Foreign', 105, 112),
 ('Office', 113, 119),
 ('Minister', 120, 128),
 (',', 128, 129),
 ('Tobias', 130, 136),
 ('Ellwood', 137, 144),
 (',', 144, 145),
 ('said', 146, 150),
 (':', 150, 151),
 ('\n\n', 151, 153),
 ('The', 153, 156),
 ('attack', 157, 163),
 ('was', 164, 167),
 ('a', 168, 169),
 ('horrific', 170, 178),
 ('act', 179, 182),
 ('of', 183, 185),
 ('terror', 186, 192),
 ('against', 193, 200),
 ('innocent', 201, 209),
 ('people', 210, 216),
 ('in', 217, 219),
 ('the', 220, 223),
 ('early', 224, 229),
 ('hours', 230, 235),
 ('of', 236, 238),
 ('Sunday', 239, 245),
 ('morning', 246, 253),
 ('in

In [325]:
from dataclasses import dataclass, field


@dataclass
class Word:
    text: str
    start_idx: int
    end_idx: int
    labels: List[str] = field(default_factory=list)

    def __post_init__(self):
        self.labels.append("O")

@dataclass
class Entity:
    text: List[Word]



In [326]:
from collections import defaultdict

def label_words(token_spans):
    words = []

    for idx, (word, start_idx, end_idx) in enumerate(token_spans):
        word = Word(word, start_idx, end_idx)

        for entity in entities:

            START = entity["begin"]
            END = entity["end"]
            TYPE = entity["type"]

            if START <= word.start_idx <= END and START <= word.end_idx <= END:
                if word.labels == ["O"]:
                    word.labels = []
                word.labels.append(TYPE)

        words.append(word)

    return words

In [327]:
words = label_words(token_spans)

[Word(text='Minister', start_idx=0, end_idx=8, labels=['Person']),
 Word(text='for', start_idx=9, end_idx=12, labels=['Person']),
 Word(text='the', start_idx=13, end_idx=16, labels=['Person']),
 Word(text='Middle', start_idx=17, end_idx=23, labels=['Person', 'Location']),
 Word(text='East', start_idx=24, end_idx=28, labels=['Person', 'Location']),
 Word(text=',', start_idx=28, end_idx=29, labels=['O']),
 Word(text='Tobias', start_idx=30, end_idx=36, labels=['Person']),
 Word(text='Ellwood', start_idx=37, end_idx=44, labels=['Person']),
 Word(text=',', start_idx=44, end_idx=45, labels=['O']),
 Word(text='responds', start_idx=46, end_idx=54, labels=['O']),
 Word(text='to', start_idx=55, end_idx=57, labels=['O']),
 Word(text='the', start_idx=58, end_idx=61, labels=['O']),
 Word(text='attack', start_idx=62, end_idx=68, labels=['O']),
 Word(text='in', start_idx=69, end_idx=71, labels=['O']),
 Word(text='the', start_idx=72, end_idx=75, labels=['Location']),
 Word(text='Karrada', start_idx=76

In [302]:
# def bio_tagger(ne_tagged: List[Tuple[str, str]]):
#     bio_tagged = []
#     prev_tag = "O"
#     for token, label in ne_tagged:
#         if label == "O":
#             bio_tagged.append((token, label))
#         elif label != "O" and prev_tag == "O":  # Begin NE
#             bio_tagged.append((token, "B-"+label))
#         elif prev_tag != "O" and prev_tag == label:  # Inside NE
#             bio_tagged.append((token, "I-"+label))
#         elif prev_tag != "O" and prev_tag != label:  # Adjacent NE
#             bio_tagged.append((token, "B-"+label))
#         prev_tag = label
#     return bio_tagged

In [328]:
from copy import deepcopy


def bio_tagger(words: List[Word]):
    max_multilabel_len = max([len(word.labels) for word in words])
    words_out = []
    prev_tag = ["O"] * max_multilabel_len
    word_iter = 0
    for word_idx, word in enumerate(words):
        bio_tagged = []
        token, labels = deepcopy(word.text), deepcopy(word.labels)

        for idx, label in enumerate(labels):
            if label == "O":
                bio_tagged.append(label)
            elif label != "O" and prev_tag[idx] == "O":  # Begin NE
                bio_tagged.append("B-"+label)
            elif prev_tag[idx] != "O" and prev_tag[idx] == label:  # Inside NE
                bio_tagged.append("I-"+label)
            elif prev_tag[idx] != "O" and prev_tag[idx] != label:  # Adjacent NE
                bio_tagged.append("B-"+label)
            prev_tag[idx] = label

        # Reset secondary/tertiary labels if no extra labels
        if len(labels) > 1:
            word_iter += 1
            if word_iter > 1:
                for idx in range(1, len(prev_tag)):
                    prev_tag[idx] = "O"
                word_iter = 0

        word.labels = bio_tagged

        words_out.append(word)

    return words
    

In [329]:
bio_tagger(words)

[Word(text='Minister', start_idx=0, end_idx=8, labels=['B-Person']),
 Word(text='for', start_idx=9, end_idx=12, labels=['I-Person']),
 Word(text='the', start_idx=13, end_idx=16, labels=['I-Person']),
 Word(text='Middle', start_idx=17, end_idx=23, labels=['I-Person', 'B-Location']),
 Word(text='East', start_idx=24, end_idx=28, labels=['I-Person', 'I-Location']),
 Word(text=',', start_idx=28, end_idx=29, labels=['O']),
 Word(text='Tobias', start_idx=30, end_idx=36, labels=['B-Person']),
 Word(text='Ellwood', start_idx=37, end_idx=44, labels=['I-Person']),
 Word(text=',', start_idx=44, end_idx=45, labels=['O']),
 Word(text='responds', start_idx=46, end_idx=54, labels=['O']),
 Word(text='to', start_idx=55, end_idx=57, labels=['O']),
 Word(text='the', start_idx=58, end_idx=61, labels=['O']),
 Word(text='attack', start_idx=62, end_idx=68, labels=['O']),
 Word(text='in', start_idx=69, end_idx=71, labels=['O']),
 Word(text='the', start_idx=72, end_idx=75, labels=['B-Location']),
 Word(text='Ka

In [330]:
import pandas as pd

df = pd.DataFrame(words)

In [331]:
df.head()

,text,start_idx,end_idx,labels
0,Minister,0,8,[B-Person]
1,for,9,12,[I-Person]
2,the,13,16,[I-Person]
3,Middle,17,23,"[I-Person, B-Location]"
4,East,24,28,"[I-Person, I-Location]"


In [332]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

In [333]:
transformed_labels = mlb.fit_transform(df["labels"])
transformed_labels

array([[0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0,

In [334]:
mlb.classes_

array(['B-Location', 'B-Organisation', 'B-Person', 'B-Temporal',
       'I-Location', 'I-Organisation', 'I-Person', 'I-Temporal', 'O'],
      dtype=object)

In [335]:
df_classes = pd.DataFrame(transformed_labels, columns=mlb.classes_)

In [336]:
df_classes.head()

,B-Location,B-Organisation,B-Person,B-Temporal,I-Location,I-Organisation,I-Person,I-Temporal,O
0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,1,0,0
4,0,0,0,0,1,0,1,0,0


In [337]:
df_new = pd.concat([df, df_classes], axis=1)
df_new.drop(["labels"], axis=1, inplace=True)
df_new.head()

,text,start_idx,end_idx,B-Location,B-Organisation,B-Person,B-Temporal,I-Location,I-Organisation,I-Person,I-Temporal,O
0,Minister,0,8,0,0,1,0,0,0,0,0,0
1,for,9,12,0,0,0,0,0,0,1,0,0
2,the,13,16,0,0,0,0,0,0,1,0,0
3,Middle,17,23,1,0,0,0,0,0,1,0,0
4,East,24,28,0,0,0,0,1,0,1,0,0
